In [1]:
# Import numpy as np
import numpy as np

# Import tensorflow as tf
import tensorflow as tf

# Import keras from tensorflow
from tensorflow import keras

# Set tensorflow seed
tf.random.set_seed(42)


In [6]:
# Import tensorflow_datasets as tfds
import tensorflow_datasets as tfds

# Load the dataset
datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)

# See the keys of datasets
print(datasets.keys())

# Get size of train and test data
train_size = info.splits["train"].num_examples
test_size = info.splits["test"].num_examples

print(train_size, test_size)


Shuffling and writing examples to /home/bhattijasjeet2429/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete8RK3E0/imdb_reviews-train.tfrecord


Shuffling and writing examples to /home/bhattijasjeet2429/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete8RK3E0/imdb_reviews-test.tfrecord


Shuffling and writing examples to /home/bhattijasjeet2429/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete8RK3E0/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /home/bhattijasjeet2429/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


dict_keys(['test', 'train', 'unsupervised'])
25000 25000


In [10]:
for X_batch, y_batch in datasets["train"].batch(2).take(1):
    for review, label in zip(X_batch.numpy(), y_batch.numpy()):
        print("Review:", review.decode("utf-8")[:200], "...")
        print("Label:", label, "= Positive" if label else "= Negative")
        print()

Review: This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting  ...
Label: 0 = Negative

Review: I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However  ...
Label: 0 = Negative



In [14]:
# Define the preprocess function
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

# Call the function
preprocess(X_batch, y_batch)


(<tf.Tensor: shape=(2, 53), dtype=string, numpy=
 array([[b'This', b'was', b'an', b'absolutely', b'terrible', b'movie',
         b"Don't", b'be', b'lured', b'in', b'by', b'Christopher',
         b'Walken', b'or', b'Michael', b'Ironside', b'Both', b'are',
         b'great', b'actors', b'but', b'this', b'must', b'simply', b'be',
         b'their', b'worst', b'role', b'in', b'history', b'Even',
         b'their', b'great', b'acting', b'could', b'not', b'redeem',
         b'this', b"movie's", b'ridiculous', b'storyline', b'This',
         b'movie', b'is', b'an', b'early', b'nineties', b'US',
         b'propaganda', b'pi', b'<pad>', b'<pad>', b'<pad>'],
        [b'I', b'have', b'been', b'known', b'to', b'fall', b'asleep',
         b'during', b'films', b'but', b'this', b'is', b'usually', b'due',
         b'to', b'a', b'combination', b'of', b'things', b'including',
         b'really', b'tired', b'being', b'warm', b'and', b'comfortable',
         b'on', b'the', b'sette', b'and', b'having', b'j

In [19]:
# Import Counter from collections
from collections import Counter



In [20]:
# Get the Counter() object vocabulary
vocabulary = Counter()


In [21]:
# Update vocabulary from train data
for X_batch, y_batch in datasets["train"].batch(2).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))


In [22]:
# Look at the 5 most common words
vocabulary.most_common()[:5]


[(b'<pad>', 63155),
 (b'the', 61137),
 (b'a', 38564),
 (b'of', 33983),
 (b'and', 33431)]

In [23]:
# Find length of vocabulary
len(vocabulary)

53893

In [27]:
# Set vocab_size to 10,000
vocab_size = 10000


In [28]:
# Extract top 10,000 most frequent words
truncated_vocabulary = [word for word, count in vocabulary.most_common()[:vocab_size]]


In [31]:
import tensorflow as tf

words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

# Example lookup
lookup_result = table.lookup(tf.constant([b"This movie was faaaaaantastic".split()]))
print(lookup_result)


tf.Tensor([[   22    12    11 10053]], shape=(1, 4), dtype=int64)


In [40]:
# Define encode_words function
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

# Apply preprocess on train data batches of size 32, repeating infinitely
train_set = datasets["train"].repeat().batch(32).map(preprocess)

# Apply encode_words to the train_set and prefetch 1 batch
train_set = train_set.map(encode_words).prefetch(1)

# Apply preprocess on test data batches of size 1000
test_set = datasets["test"].batch(1000).map(preprocess)

# Apply encode_words to the test_set
test_set = test_set.map(encode_words)

# Preview first batch from train_set
for X_batch, y_batch in train_set.take(1):
    print(X_batch)
    print(y_batch)


tf.Tensor(
[[  22   11   28 ...    0    0    0]
 [   6   21   70 ...    0    0    0]
 [4099 6881    1 ...    0    0    0]
 ...
 [  22   12  118 ...  331 1047    0]
 [1757 4101  451 ...    0    0    0]
 [3365 4392    6 ...    0    0    0]], shape=(32, 60), dtype=int64)
tf.Tensor([0 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 1 1 1 0 1 1 1 1 1 0 0 0 1 0 0 0], shape=(32,), dtype=int64)


In [45]:
embed_size = 128

model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True,
                           input_shape=[None]),
    keras.layers.GRU(4, return_sequences=True),
    keras.layers.GRU(2),
    keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [49]:
import time

start = time.time()

model.fit(train_set, steps_per_epoch=train_size // 32, epochs=2)

end = time.time()

print("Time of execution:", end - start)

model.evaluate(test_set)


Epoch 1/2
781/781 [==============================] - 91s 107ms/step - loss: 0.5910 - accuracy: 0.6626
Epoch 2/2
781/781 [==============================] - 81s 104ms/step - loss: 0.3785 - accuracy: 0.8385
Time of execution: 172.42676210403442
25/25 [==============================] - 7s 155ms/step - loss: 0.5338 - accuracy: 0.7560


[0.5337878465652466, 0.7559599876403809]